In [2]:
deployment = "gpt4"
embedding_deployment = "embedding"
model = "gpt4"

In [2]:
!pip install langchain_experimental

In [17]:
def get_current_cluster_state(cluster_name):
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

In [27]:
from langchain.agents import initialize_agent, Tool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMRequestsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import time

llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)
tools = [
    Tool(
        name="Get current cluster state", func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given service cluster, 
        And the input of the function is only one parameter, the service name, for example, to get the state of ADX service, 
        the input for invoking the function is "ADX"         
        """)
]

planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)


In [32]:
agent.run("Check the service named 'DSP', if there's something wrong with the service, give me some suggestions to fix it.")



> Entering new PlanAndExecute chain...
steps=[Step(value="Identify the service named 'DSP'."), Step(value="Check the status of the 'DSP' service."), Step(value='If the service is running correctly, inform the user that there is no issue with the service.'), Step(value='If the service is not running correctly, diagnose the problem.'), Step(value='Based on the diagnosis, suggest possible solutions to fix the issue.'), Step(value="Given the above steps taken, respond to the user's original request by providing the status of the 'DSP' service and any necessary solutions to fix it, if applicable.\n")]

> Entering new AgentExecutor chain...
Thought: The user has not provided the name of the DSP service yet. I need to ask for it again to proceed with the task.
Action:
```
{
  "action": "Final Answer",
  "action_input": "Could you please specify the name of the DSP service that you want to check?"
}
```

> Finished chain.
*****

Step: Identify the service named 'DSP'.

Response: Could you pl

'There seems to be an issue with pulling the latest image for the DSP service. This could be due to network issues, incorrect image name, or the image not being available. I suggest you to verify the image name and availability. If the problem persists, you may need to check your network connection or contact your system administrator.'

# 文本向量化（embedding)

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
# embedding = OpenAIEmbeddings() #如果直接使用OpenAI的GPT服务
embedding = OpenAIEmbeddings(deployment=embedding_deployment) #deployment是你在Azure中的 embedding 模型的部署名字
sentence1 = "我是一名软件工程师"
sentence2 = "小张从事法律工作"
sentence3 = "我是一名程序员"

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [4]:
!pip install numpy

In [4]:
import numpy as np
print(np.dot(embedding1,embedding2))
print(np.dot(embedding2,embedding3))
print(np.dot(embedding1,embedding3))


0.7987662070931474
0.7992461569419606
0.9592105615669977


# 处理大文本

## 切割

In [5]:
def split_file_into_chunks(file_path, chunk_size):
    chunks = []
    with open(file_path, 'r') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            chunks.append(chunk)
    return chunks

In [6]:
split_file_into_chunks("spotmax_intro.txt",100)

['SpotMax\n打破成本约束，释放云端创造力\n\n构建大规模云端系统的挑战\n云计算平台“按需获取”的特性使得用户可以随时获取所需计算资源，实现计算资源与业务规模的动态匹配。云计算时代计算资源已不再是构建',
 '大规模系统的瓶颈，多数情况下成本成为了企业在构建大规模应用时的最大挑战。\n\n什么是 SpotMax?\nSpotMax是一个利用弹性计算资源进行云端系统成本优化的体系（包括相关产品及解决方案）。Spot',
 'Max旨在帮助用户使用云平台提供的弹性资源（如：AWS Spot Instance）在保证系统可用性，甚至获得更高可用性的前提下，实现成本的最大化节省。SpotMax来自于Mobvista降低成本的成',
 '功实践。\nSpotMax由以下三个部分组成：\nMaxArch：构建高可用性，高弹性（Resilient）架构，可以允许系统在不影响业务的前提下更充分的使用具有成本优势的高弹性计算资源（Spot Ins',
 'tance）。\nMaxChaos：利用混沌工程可以有效的评估系统的容错能力，其评估结果将成为MaxGroup自动规划合理弹性集群的依据。实现MaxArch的持续改进及评估。\nMaxGroup：根据服务',
 '工作模式和运行特征，优化主机群集。智能弹性集群管理可以有效减少弹性计算资源波动（如：竞价实例的中断）带来的集群服务能力变化。并在面向成本优化构建的混合机型集群最大发挥集群不同机型的服务能力\n以上各部分',
 '将以咨询服务和软件产品的形式提供。\nMaxGroup 核心功能\nMaxGroup可以根据服务的工作模式和运行时特征，对主机集群的组成提出优化建议（如预留实例、按需实例与Spot型实例的比率，Spot实',
 '例池等），并估算出此优化节省的成本。\n用户确认优化集群方案后，自动实现新集群的构建或原有集群的构成的直接优化(安全替换现有集群的运行实例)。\n通过对集群中Spot instance中断的预先补偿, 降',
 '低中断带来的服务器容量变化带来的服务能力影响，实现服务及数据的提前迁移。\nMaxGroup将根据现货价格和终止率的变化持续优化集群结构。 \n支持智能的动态负载均衡，由于集群为多机型混合，不同机型的处理',
 '能力有差异，智能的动态负载均衡将实现按处理能力的负载分配，实现集群资源

## 向量化 

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings

def embed(chunks): 
    # embedding = OpenAIEmbeddings()#如果直接使用OpenAI的GPT服务
    embedding = OpenAIEmbeddings(deployment=embedding_deployment)
    return [embedding.embed_query(chunk) for chunk in chunks]
        

In [10]:
chunks = split_file_into_chunks("spotmax_intro.txt",100)
embeddeds = embed(chunks)

## 向量检索

In [11]:
def find_k_largest_indices(input_list, k):
    sorted_indices = sorted(range(len(input_list)), key=lambda i: input_list[i], reverse=True)
    print(input_list)
    print(sorted_indices)
    return sorted_indices[:k]

def search(chunks,embeddeds,top_k, txt):
    embedding = OpenAIEmbeddings(deployment="embedding",chunk_size=1)
    embedded_txt = embedding.embed_query(txt)
    distances = [np.dot(embedded_txt,embedded) for embedded in embeddeds]
    ret_idx = find_k_largest_indices(distances, top_k)
    return [chunks[i] for i in ret_idx]

In [12]:
search(chunks,embeddeds,2, "提高系统可用性")

[0.8023933391562698, 0.8125168324498859, 0.7812473775874645, 0.817075818463568, 0.7898890421834843, 0.8397854867756218, 0.7722375027327882, 0.7687244419032947, 0.8104800333965131, 0.8089354290813963, 0.8057141418303226]
[5, 3, 1, 8, 9, 10, 0, 4, 2, 6, 7]


['工作模式和运行特征，优化主机群集。智能弹性集群管理可以有效减少弹性计算资源波动（如：竞价实例的中断）带来的集群服务能力变化。并在面向成本优化构建的混合机型集群最大发挥集群不同机型的服务能力\n以上各部分',
 '功实践。\nSpotMax由以下三个部分组成：\nMaxArch：构建高可用性，高弹性（Resilient）架构，可以允许系统在不影响业务的前提下更充分的使用具有成本优势的高弹性计算资源（Spot Ins']

In [13]:
search(chunks,embeddeds,2, "省钱")

[0.7626223356791848, 0.7574077611525133, 0.7236185893772494, 0.7525226844839109, 0.6965344275382886, 0.748106643696892, 0.7173777528336387, 0.7403726949080747, 0.7204675497717147, 0.7216070546204, 0.7264813507926091]
[0, 1, 3, 5, 7, 10, 2, 9, 8, 6, 4]


['SpotMax\n打破成本约束，释放云端创造力\n\n构建大规模云端系统的挑战\n云计算平台“按需获取”的特性使得用户可以随时获取所需计算资源，实现计算资源与业务规模的动态匹配。云计算时代计算资源已不再是构建',
 '大规模系统的瓶颈，多数情况下成本成为了企业在构建大规模应用时的最大挑战。\n\n什么是 SpotMax?\nSpotMax是一个利用弹性计算资源进行云端系统成本优化的体系（包括相关产品及解决方案）。Spot']

In [14]:
search(chunks,embeddeds,2, "什么是MaxGroup")

[0.7592167264242349, 0.7836962526531055, 0.7419871546836885, 0.7720463741599968, 0.8115075334429604, 0.7319843405717201, 0.8144565372167445, 0.6795742540190728, 0.8217077990486893, 0.8230605403147829, 0.7877589772115714]
[9, 8, 6, 4, 10, 1, 3, 0, 2, 5, 7]


['能力有差异，智能的动态负载均衡将实现按处理能力的负载分配，实现集群资源利用的最大化。\nMaxGroup可以根据工作负载变化模式（缓慢变化或急剧变化）和业务需求为系统自动伸缩选择合适的实例类型。 \nMa',
 '低中断带来的服务器容量变化带来的服务能力影响，实现服务及数据的提前迁移。\nMaxGroup将根据现货价格和终止率的变化持续优化集群结构。 \n支持智能的动态负载均衡，由于集群为多机型混合，不同机型的处理']

In [15]:
import openai
import numpy as np
def anwser_question_with_doc(question, chunks, embeddeds):
    relevent_chunks = search(chunks,embeddeds,2, question)
    prompt = """
    仅通过总结以下的文字片段回答用户问题, 注意保持回答的语言通顺（字数在30字以内）
    --- 
    """
    for rchunks in relevent_chunks:
        prompt = prompt + "\n'" + rchunks + "'"
    
    response = openai.ChatCompletion.create(
    engine=deployment, # engine = "deployment_name".
    messages=[
            {"role": "system", "content": prompt},   
            {"role": "user", "content": question}
        ],
        temperature = 0.9, 
        max_tokens = 200
   )
    return response.choices[0].message.content

In [16]:
print(anwser_question_with_doc("如何提高可用性", chunks,embeddeds))

[0.785118591499548, 0.7976991410829841, 0.7675298917649545, 0.8065815459820063, 0.7816592248948016, 0.8356628134875835, 0.7624604542313403, 0.765247532222735, 0.8019779315035528, 0.8071677645976287, 0.7917167790813946]
[5, 9, 3, 8, 1, 10, 0, 4, 2, 7, 6]
通过使用智能弹性集群管理优化主机群集和实现动态负载均衡，可以最大化集群资源利用，从而提高可用性。


In [17]:
print(anwser_question_with_doc("SpotMax是什么", chunks,embeddeds))

[0.8770056249839784, 0.8362516140610949, 0.7834500483012314, 0.8301407011787743, 0.7506028306598755, 0.7373641089558522, 0.7637066032721377, 0.7063620631436798, 0.752122314461898, 0.7571780038817336, 0.7157490897923696]
[0, 1, 3, 2, 6, 9, 8, 4, 5, 10, 7]
SpotMax是一个利用弹性计算资源进行云端系统成本优化的体系，包括相关产品及解决方案。


In [32]:
print(anwser_question_with_doc("如何防止spot instance中断", chunks,embeddeds))

[0.7849296999456946, 0.7818371614098701, 0.7384494647990351, 0.7937139964158784, 0.7176644252472997, 0.7744640165796035, 0.7446397380953562, 0.7997561198477201, 0.7497060465878663, 0.7364696122979241, 0.7572099874977832]
[7, 3, 0, 1, 5, 10, 8, 6, 2, 9, 4]
通过对集群中Spot instance中断的预先补偿，防止其中断。
